# Implementation of different versions of the benefit difference fairness function

In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import numpy as np
import multiprocessing as mp
from src.util import sigmoid
from src.feature_map import IdentityFeatureMap
from src.functions import cost_utility, demographic_parity
from src.plotting import plot_results_over_lambdas
from src.training import train_multiple

## The parameters used by the the original authors  

In [ ]:
bias = True
dim_x = 1
dim_theta = dim_x + 1 if bias else dim_x

lambdas = np.logspace(1, 5, base=10, endpoint=True, num=5)
#lambdas = np.insert(arr=lambdas, obj=0, values=[0.0])

def util_func(**util_params):
    util = cost_utility(cost_factor=0.55, **util_params)
    return util

training_parameters = {    
    'model':{
        'theta': [-3.5, 0.6],
        'benefit_value_function': demographic_parity,
        'utility_value_function': util_func,
        'feature_map': IdentityFeatureMap(dim_theta),
        'keep_collected_data': False,
        'use_sensitve_attributes': False,
        'bias': bias
    },
    'optimization': {
        'time_steps':200,
        'epochs': 32,
        'batch_size':512,
        'learning_rate': 0.5,
        'decay_rate': 0.8,
        'decay_step': 30,
        'fairness_rates': lambdas
    },
    'data': {
        'fraction_protected':0.5,
        'num_test_samples': 2000,
        'num_decisions': 32 * 512
    }
}


## Correct Benefit Difference

In [ ]:
def fairness_function(**fairness_kwargs):
    policy = fairness_kwargs["policy"]
    x = fairness_kwargs["x"]
    s = fairness_kwargs["s"]
    y = fairness_kwargs["y"]
    decisions = fairness_kwargs["decisions"]
    ips_weights = fairness_kwargs["ips_weights"]

    benefit = policy.benefit_function(decisions=decisions, y=y)
    log_gradient = policy._log_gradient(x, s)
    benefit_grad = benefit * log_gradient

    if ips_weights is not None:
        benefit *= ips_weights
        benefit_grad *= ips_weights
        
    # benefit-difference * grad-benefit-difference
    return policy._mean_difference(benefit, s) * policy._mean_difference(benefit_grad, s)

training_parameters["model"]['fairness_function'] = fairness_function

results = train_multiple(training_parameters, iterations=20, verbose=True, asynchronous=False)
plot_results_over_lambdas(
    results["utility_stats"]["mean"], 
    results["utility_stats"]["stddev"], 
    results["benefit_delta_stats"]["mean"], 
    results["benefit_delta_stats"]["stddev"], 
    lambdas, 
    file_path="results.png")